In [1]:
import pandas as pd
import numpy as np 
import datetime
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
import optuna.integration.lightgbm as lgb_o

In [2]:
class Results:
    @staticmethod
    def scrape(race_id_list):
        
        #race_id をkeyにしてdataframe型を格納
        race_results = {}
        for race_id in tqdm(race_id_list):
            time.sleep(0.5)
            try:
                url = "https://db.netkeiba.com/race/" + race_id
                #メインになるテーブルデータを取得
                df = pd.read_html(url)[0]
                
                html = requests.get(url)
                html.encoding = "EUC-JP"
                soup = BeautifulSoup(html.text, "html.parser")
                
                #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                texts = (
                    soup.find("div" ,attrs={"class": "data_intro"}).find_all("p")[0].text
                    + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                )
                info = re.findall(r'\w+', texts)
                for text in info:
                    if text in ["芝", "ダート"]:
                        df ["race_type"] = [text] * len(df)
                    if "障" in text:
                        df ["race_type"] = ["障害"] * len(df)
                    if "m" in text:
                        df ["cource_len"] = [int(re.findall(r"\d+", text)[0])] * len(df)
                    if text in ["良","稍重","重","不良"]:
                        df ["ground_state"] = [text] * len(df)
                    if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                        df ["weather"] = [text] * len(df)
                    if "年" in text:
                        df ["data"] = [text] *len(df)
                    
                #馬ID、騎手IDをスクレイピング
                horse_id_list = []
                horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/horse")}
                )
                for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])
                
                jockey_id_list = []
                jockey_a_list = soup.find("table", attrs={"summary":"レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
                for a in jockey_a_list:
                    jockey_id = re.findall(r"\d+", a["href"])
                    jockey_id_list.append(jockey_id[0])
                
                df["horse_id"] = horse_id_list
                df["jockey_id"] = jockey_id_list
                
                #インデックスをrace_idにする
                df.index = [race_id] * len(df)
                
                race_results[race_id] = df
                
            #存在しないrace_idを飛ばす
            except IndexError:
                continue
            #中断された時にデータを返せるようにする
            except Exception as e:
                print(e)
                break
            #jupyterの停止をした場合
            except:
                break
                
        
        #pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])
        
        return race_results_df
                

                        
    

In [3]:
race_id_list = []
for place in range(1,11,1):
    for kai in range(1,13,1):
        for day in range(1,13,1):
            for r in range(1,13,1):
                race_id = "2019" + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)
                
results = Results.scrape(race_id_list)

  0%|          | 0/17280 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [44]:
#与える
results.to_pickle('results.pickle')

In [32]:
#吸い出す
results = pd.read_pickle('results.pickle')

In [6]:
results

NameError: name 'results' is not defined

In [41]:
Results.scrape(race_id_list)

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,cource_len,weather,race_type,ground_state,data,horse_id,jockey_id
201901010101,1,1,1,ゴルコンダ,牡2,54,ルメール,1:48.3,NaN,1.4,1.0,518(-16),[東] 木村哲也,1800,曇,芝,良,2019年7月27日,2017105318,05339
201901010101,2,3,3,プントファイヤー,牡2,54,岩田康誠,1:50.1,大,3.5,2.0,496(-8),[東] 手塚貴久,1800,曇,芝,良,2019年7月27日,2017104612,05203
201901010101,3,4,4,ラグリマスネグラス,牡2,51,団野大成,1:50.9,5,46.6,6.0,546(+6),[東] 藤沢和雄,1800,曇,芝,良,2019年7月27日,2017103879,01180
201901010101,4,8,9,キタノコドウ,牡2,51,菅原明良,1:51.5,3.1/2,56.8,7.0,458(-8),[東] 高木登,1800,曇,芝,良,2019年7月27日,2017106259,01179
201901010101,5,5,5,ネモフィラブルー,牡2,54,川島信二,1:51.7,1.1/2,140.3,9.0,436(0),[西] 矢作芳人,1800,曇,芝,良,2019年7月27日,2017104140,01062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201901020612,10,7,11,クアトレフォイル,牡4,57,石川倭,1:45.7,2.1/2,224.2,14.0,498(+6),[東] 稲垣幸雄,1700,晴,ダート,良,2019年9月1日,2015103320,05534
201901020612,11,8,13,ベイコート,牝3,52,石川裕紀,1:45.8,1/2,4,1.0,500(+10),[西] 高橋義忠,1700,晴,ダート,良,2019年9月1日,2016101169,01150
201901020612,12,6,9,スーパーライナー,牡6,57,坂井瑠星,1:46.4,3.1/2,15.5,8.0,486(-8),[西] 今野貞一,1700,晴,ダート,良,2019年9月1日,2013105629,01163
201901020612,13,3,3,レジェンディスト,牝4,55,加藤祥太,1:46.9,3,8.9,5.0,474(+18),[西] 庄野靖志,1700,晴,ダート,良,2019年9月1日,2015104475,01156
